In [26]:
# Устанавливаем необходимые библиотеки
!pip install tensorflow opencv-python

# Монтируем Google Drive
from google.colab import drive
drive.mount('/content/drive')

import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import cv2
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import numpy as np

# Указываем путь к директории с данными на Google Drive
data_dir = '/content/drive/MyDrive/color_dataset'  # Заменить на свой путь к папке с изображениями

# Проверка, существует ли папка с данными
if not os.path.exists(data_dir):
    print(f"Папка {data_dir} не найдена. Проверьте путь к папке.")
else:
    print(f"Папка {data_dir} найдена. Доступ к данным разрешен.")

    # Используем ImageDataGenerator для загрузки и аугментации данных
    train_datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)

    # Загрузка тренировочных данных
    train_generator = train_datagen.flow_from_directory(
        data_dir,
        target_size=(128, 128),
        batch_size=32,
        class_mode='sparse',  # Для многоклассовой классификации
        subset='training'
    )

    # Загрузка данных для валидации
    validation_generator = train_datagen.flow_from_directory(
        data_dir,
        target_size=(128, 128),
        batch_size=32,
        class_mode='sparse',  # Для многоклассовой классификации
        subset='validation'
    )

    # Создание модели нейронной сети
    def create_model():
        model = models.Sequential([
            layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(64, (3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(128, (3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            layers.Flatten(),
            layers.Dense(128, activation='relu'),
            layers.Dense(train_generator.num_classes, activation='softmax')  # Классы равны количеству цветов
        ])
        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        return model

    # Создаем модель
    model = create_model()


    # Сохраняем модель
    model.save('/content/drive/MyDrive/color_classifier.h5')

    # Функция для предсказания цвета изображения
    def predict_color(model, image_path):
    # Загружаем и подготавливаем изображение с использованием PIL
      img = Image.open(image_path)

    # Преобразуем изображение в формат RGB, если оно в другом формате (например, RGBA)
      img = img.convert('RGB')

      img = img.resize((128, 128))  # Изменяем размер изображения
      img = np.array(img) / 255.0  # Нормализуем изображение

    # Добавляем размерность для пакета (batch)
      img = np.expand_dims(img, axis=0)

    # Прогнозируем
      prediction = model.predict(img)
      class_idx = np.argmax(prediction)  # Индекс с наибольшей вероятностью

    # Получаем метки классов (цвета)
      class_labels = list(train_generator.class_indices.keys())  # Метки классов
      return class_labels[class_idx]

    # Прогнозирование для изображения (замени путь на нужное изображение)
    image_path = '/content/drive/MyDrive/4.jpeg'  # Заменить на путь к изображению
    predicted_color = predict_color(model, image_path)
    print(f'Основной цвет изображения: {predicted_color}')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Папка /content/drive/MyDrive/color_dataset найдена. Доступ к данным разрешен.
Found 1803 images belonging to 12 classes.
Found 447 images belonging to 12 classes.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
Основной цвет изображения: Red


In [23]:
 # Обучение модели
model.fit(train_generator, epochs=10, validation_data=validation_generator)

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


57/57 ━━━━━━━━━━━━━━━━━━━━ 80s 1s/step - accuracy: 0.3992 - loss: 1.6579 - val_accuracy: 0.6331 - val_loss: 0.9838
Epoch 2/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 66s 1s/step - accuracy: 0.6716 - loss: 0.8532 - val_accuracy: 0.6667 - val_loss: 0.8442
Epoch 3/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 67s 1s/step - accuracy: 0.7322 - loss: 0.6837 - val_accuracy: 0.7136 - val_loss: 0.7937
Epoch 4/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 67s 1s/step - accuracy: 0.7766 - loss: 0.6180 - val_accuracy: 0.6600 - val_loss: 0.8847
Epoch 5/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 67s 1s/step - accuracy: 0.7588 - loss: 0.5980 - val_accuracy: 0.7494 - val_loss: 0.7076
Epoch 6/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 67s 1s/step - accuracy: 0.7932 - loss: 0.4979 - val_accuracy: 0.7651 - val_loss: 0.6939
Epoch 7/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - accuracy: 0.7897 - loss: 0.5585 - val_accuracy: 0.7114 - val_loss: 0.7340
Epoch 8/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 65s 1s/step - accuracy: 0.7981 - loss: 0.4846 - val_accuracy: 0.8009 - val_loss: 0.6201
Epo